In [1]:
# load the new bank_tract_census_CLEAN
import matplotlib.pyplot as plt
from matplotlib  import rc
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import numpy as np
import os
bank_tract = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [2]:
# Get the Arizona tract shapefile and convert to UTM Zone 17N coordinate system
shape_az = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_04_tract.zip").to_crs(epsg=32617)
shape_ca = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_06_tract.zip").to_crs(epsg=32617)
shape_all = pd.concat([shape_az, shape_ca], ignore_index=True)

In [3]:
shape_all['GEOID'] = shape_all['GEOID'].astype(int)
bank_tract['census_tract'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13837 entries, 0 to 13836
Series name: census_tract
Non-Null Count  Dtype
--------------  -----
13837 non-null  int64
dtypes: int64(1)
memory usage: 108.2 KB


In [4]:
# merge the shape file with bank_tract
shape_all['census_tract'] = shape_all['GEOID']
bank_tract = bank_tract.merge(shape_all, how = 'left', on = ['census_tract'],
                              indicator = True, validate= 'm:1') 

In [5]:
# print(list(bank_tract.columns))

In [6]:
cities = ['phx', 'tcs', 'sf', 'la']
phx_zoom = {'x': [-2.5e6,-2.35e6], 'y' : [4.1e6,4.25e6]}
tcs_zoom = {'x': [-1.17e7, -1.16e7], 'y': [3.63e6, 3.65e6]}
sf_zoom = {'x': [-1.362e7, -1.355e7], 'y': [4.33e6, 4.37e6]}
la_zoom = {'x': [-1.287e7, -1.275e7], 'y': [4.033e6, 4.086e6]}
citieszoom = [phx_zoom, tcs_zoom, sf_zoom, la_zoom]

In [8]:
#denial_rate, credit_spread, np.log(# loans), np.log($ loans), avg LTV 
bank_tract ["white_rate"] = (bank_tract ["WhitePop"]) / bank_tract ["Tot.Pop"] * 100
bank_tract ['majority-minority'] = bank_tract ["white_rate"] < 50
bank_tract ['below_p10_income'] = bank_tract ["Med.HousehIncome"] < bank_tract["Med.HousehIncome"].quantile(0.1)
bank_tract ['high_hispanic'] = (bank_tract ['HispanicLatinoPop'] / bank_tract['Tot.Pop']) > 0.5
bank_tract ['high_black'] = (bank_tract ['BlackPop'] / bank_tract['Tot.Pop']) > 0.5
bank_tract ['high_asian'] = (bank_tract ['AsianPop'] / bank_tract['Tot.Pop']) > 0.5
bank_tract ['high_white'] = (bank_tract ['WhitePop'] / bank_tract['Tot.Pop']) > 0.5

bank_tract ['log(numloans)'] = np.log(bank_tract['num_approved_loans'])
bank_tract ['log(dol_loans)'] = np.log(bank_tract['avg_approved_loan_size'])


listofshading = ['denial_rate', 'mean_approved_rate_spread', 'log(numloans)', 'log(dol_loans)', 'mean_LTV']

#majority_minority, income_below_p10, high hispanic, high Black, high Asian
listofredlining = ['majority-minority', 'below_p10_income', 'high_hispanic', 'high_black', 'high_asian', 'high_white']

In [10]:
def red_line_map(df,shading_var:str,redline_var:str,optional_title= None,option_zoom_params=None):
    fig, ax = plt.subplots(1, 1, figsize = (20, 10))

    # Plot data
    # Source: https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html
    df.plot(column = shading_var,
                           ax = ax,
                           cmap = "Greens",
                           legend = True)

    # add the the majority minority tract lines in red
    query = f'`{redline_var}`'
    df.query(query).boundary.plot(color='red',ax=ax, linewidth=2)

    # add the other tract lines in grey
    query = f'`{redline_var}` == False'    
    df.query(query).boundary.plot(color='gainsboro',ax=ax, linewidth=.2)

    # Stylize plots
    plt.style.use('bmh')

    # Set title
    if optional_title:
        plt.title(f'{shading_var} in AZ \n Red outlines are based on {redline_var}', fontdict = {'fontsize': 15})
    # optionally, zoom in
    if option_zoom_params:
        ax.set_xlim(option_zoom_params['x'])
        ax.set_ylim(option_zoom_params['y'])

In [9]:
# for city in citieszoom: 
#     city_zoom = zoom_param[city]
#     for shade in listofshading:
#         for redline in listofredlining:
#             red_line_map(bank_tract, shade, redline, True, city_zoom)
            # clear plot
            #f () ---> filename = f'plots/{city[0:3]}-{shade[0:10]}-{redline[0:10]}.csv
            # save plot --> plots/

NameError: name 'zoom_param' is not defined